## **To Do**

**Page 0** 


1. Title
2. No. Name and Reservation Status of AC 
3. No. Name and Reservation Status of PC 
4. Revision Details <br> 
> 1.   Year
> 2. Qualifying Date
> 3. Type of Revision
> 4. Date of Publication

5. Roll Identification

6. Part and Polling Areas Details <br> 
> 1. No. and name of sections in the part
> 2. முக்கிய நகரம்‌ (அ) கிராமம்‌ (Town/Village) 
> 3.கி.நி.அ மண்டலம்‌ (Mandal)
> 4. பிர்கா (Firkah/Revenue Division)
> 5. காவல்‌ நிலையம்‌ (Police Station)
> 6. வட்டம்‌ (Circle)
> 7. மாவட்டம்‌ (District)
> 8. அஞ்சலகக்குறியீட்டெண் (Pincode)

7. Polling Station Details <br> 
> 1. No. and name of PS
> 2. Address of PS 
> 3. Type of PS
> 4. Auxilliary PS Number

7. Number of Electors <br> 
> 1. Start Number
> 2. Ending Number 
> 3. Net Electors (M/F/TG/Tot)



**Page 1** 

1. None - Remove Page
2. Crop PS Image and Store


**Page 2** 

1.   Voter Block <br> 
> 1. VID
> 1. Name 
> 2. Father/Husband Name 
> 3. House Number
> 4. Age


 2.   Age on, Publication Date and PageNO

**Page 3 - n** 


1.   Voter Block <br> 
> 1. VID
> 1. Name 
> 2. Father/Husband Name 
> 3. House Number
> 4. Age


 2.   Age on, Publication Date and PageNO



In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pdf2image
!sudo apt-get install poppler-utils
!pip install PyMuPDF

!apt-get install tesseract-ocr-tam
!ls -ltr /usr/share/tesseract-ocr/4.00/tessdata/tam.traineddata
!tesseract -v

In [ ]:
from pdf2image import convert_from_path
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image
import pdf2image
import difflib
import re
import pandas as pd
import sys
import numpy as np
%cd /content/drive/MyDrive/econ_polling/scraping_tamil_pdf/

#!ls

/content/drive/MyDrive/econ_polling/scraping_tamil_pdf


In [ ]:
###Function to convert PDFs to Images 
# Input: PDF File name with .pdf and folder relative to current directory
# Output: Saves images in the folder

def image_pdf(pdf_file, folder):
  #pdf_file='test'
  images = convert_from_path(pdf_file)
  if not os.path.exists(folder):
      os.makedirs(folder)
  for i in range(len(images)):

    images[i].save(folder+'/'+ str(i) +'.jpg', 'JPEG')
  return images 

###Function to convert Images to Text 
# Input: Image File with and folder relative to current directory
# Output: Saves images in the folder

def image_text(imageORfile,psm=11,if_file=True):

  #imageORfile='test0.jpg'
  if if_file:
    text_extracted = pytesseract.image_to_string(Image.open(imageORfile),config='psm '+str(psm),lang='tam+eng')
  else:
    text_extracted = pytesseract.image_to_string(imageORfile,config='psm '+str(psm), lang='tam+eng') 
  return text_extracted 

###Function to crop images 


def image_crop(imageORfile,crop_dim_list,if_file=True):
  if if_file:
    im=Image.open(imageORfile)
  else:
    im= imageORfile
  im1 = im.crop(crop_dim_list)
  return im1 

###Function to crop a page into voter blocks

def voter_block_crop(page_image,is_page2=False):
  crop_block_list=[]
  imag=page_image
  if is_page2:
    start_height=133
  else:
    start_height=115
  width=501
  height=195
  b=start_height
  for i in np.arange(1,31,1):  
    if (i%3)==1:
      a=60
      c,d=a+width,b+height
      crop_list=[a,b,c,d]
      crop_block_list.append(image_crop(imag,crop_list,False))
    if (i%3)==2:
      a=561
      c,d=a+width,b+height
      crop_list=[a,b,c,d]
      crop_block_list.append(image_crop(imag,crop_list,False))    
    if (i%3)==0:
      a=1063
      c,d=a+width,b+height
      crop_list=[a,b,c,d]
      if i>=3:
        b+=height
      crop_block_list.append(image_crop(imag,crop_list,False))
  crop_block_image_list=crop_block_list
  
  return crop_block_image_list

#### Voter ID  ####
def get_voterid(text_rows):
  row = text_rows
  if (len(row))>0:
    voterid=row[0].strip()
  return voterid

#### Gender and Age ####

def get_gender_age(text_rows):
  row = text_rows

  if (len(row))>0:
    for row_iter in range(len(row)):
      if row[row_iter].startswith(('வயது')):
        gender = row[row_iter].replace('-', '').replace('Available','').replace('is','').replace('Photo','').split(':')[-1]
        age = row[row_iter].replace('-', '').replace('Available','').replace('is','').replace('Photo','').split(('பாலினம்'))[0].strip('வயது:')       
  return age, gender

#### Father/Husband Name ####

def get_f_h_name(text_rows):
  row = text_rows
  if (len(row))>0:    
    for row_iter in range(len(row)):
      if row[row_iter].startswith(('கணவர்‌','தந்‌ைத','தந்த')):
        if row[row_iter].startswith(('தந்‌ைத','தந்த')):
          if not row[row_iter+1].startswith(('வீட்டு')):
            f_h_name=row[row_iter]+' '+row[row_iter+1].replace('-', '').replace('Available','').replace('is','').replace('Photo','').strip('தந்‌ைத பெயர்‌:')
          else:
            f_h_name=row[row_iter].replace('-', '').replace('Available','').replace('is','').replace('Photo','').strip('தந்‌ைத பெயர்‌:')

          f_or_h='father'
        elif row[row_iter].startswith(('கணவர்‌')):
          if not row[row_iter+1].startswith(('வீட்டு')):
            f_h_name=row[row_iter]+' '+row[row_iter+1].replace('-', '').replace('Available','').replace('is','').replace('Photo','').strip('கணவர்‌ பெயர்‌:')
          else:
            f_h_name=row[row_iter].replace('-', '').replace('Available','').replace('is','').replace('Photo','').strip('கணவர்‌ பெயர்‌:')


          f_or_h='husband'
        else:
          f_or_h=''
  return f_or_h,f_h_name

#### Name ####
def get_name(text_rows):
  row = text_rows
  if (len(row))>0:
    for row_iter in range(len(row)):
      if row[row_iter].startswith('பெயர்‌:'):
        if not row[row_iter+1].startswith(('கணவர்‌','தந்‌ைத','தந்த')):
          name=row[row_iter]+' '+row[row_iter+1].replace('-', '').strip('பெயர்‌:')
        else:
          name=row[row_iter].replace('-', '').strip('பெயர்‌:')
  return name

#### Get Details from Voter Block #####

def get_voter_block_textrows(block_image):
  text_raw= image_text(block_image,psm=11,if_file=False)
  row=[]
  slno=False
  for j in text_raw.split('\n'):
    if j != '' and j != ' ' and j != '\x0c'and j != '  ' and j != '   'and j != '    ':
      row.append(j)
  return row

def get_voter_block_details(row):
  v_id=get_voterid(row)
  v_name=get_name(row)
  f_or_h,f_h_name=get_f_h_name(row)
  age, gender = get_gender_age(row)
  slno=True
  return v_id,v_name,f_or_h,f_h_name,age,gender,slno


In [ ]:
sl_no=1
lista=[]
for page_iter in tqdm(range(len(image_list))):
    
    if page_iter==2:
       block_images=voter_block_crop(image_list[page_iter],True)
    elif page_iter>2:
       block_images=voter_block_crop(image_list[page_iter],False)
    else:
      continue
    for block_iter in range(len(block_images)):
      rows_text=get_voter_block_textrows(block_images[block_iter])
      if len(rows_text)>2:
        v_id,v_name,f_or_h,f_h_name,age,gender,slno=get_voter_block_details(rows_text)
        #print(v_id)
        print(sl_no)
        lista.append([v_id,v_name,f_or_h,f_h_name,age,gender,sl_no])
        sl_no+=1



    





  0%|          | 0/37 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
asw=pd.DataFrame(lista,columns=['v_id','v_name','f_or_h','f_h_name','age','gender','slno'])
asw.head(30)

